<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_MathChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 자동 생성된 상담원 채팅: 수학 문제 해결을 위한 MathChat 활용하기

AutoGen은 자동화된 채팅을 통해 공동으로 작업을 수행하는 데 사용할 수 있는 LLM, 도구 또는 사람 기반의 대화형 에이전트를 제공합니다. 이 프레임워크는 다중 에이전트 대화를 통해 도구 사용과 사람 참여를 허용합니다. 이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)에서 확인할 수 있습니다.

MathChat은 수학 문제 해결을 위한 실험적인 대화 프레임워크입니다. 이 노트북에서는 수학 문제를 풀기 위해 MathChat을 사용하는 방법을 보여드립니다. MathChat은 다른 노트북(예: [코드 생성, 실행 및 디버깅을 통한 자동화된 작업 해결](https://github.com/microsoft/autogen/blob/main/notebook/agentchat_auto_feedback_from_code_execution.ipynb)에서 `AssistantAgent`와 `MathUserProxyAgent`의 사용법과 유사하게 `AssistantAgent`와 `UserProxyAgent`를 사용합니다.) 기본적으로 `MathUserProxyAgent`는 MathChat 프롬프트에 대응하는 다른 자동 응답 메커니즘을 구현합니다. 자세한 내용은 논문 [GPT-4를 이용한 수학 문제 해결에 대한 실증적 연구](https://arxiv.org/abs/2306.01337) 또는 [블로그 포스트](https://microsoft.github.io/autogen/blog/2023/06/28/MathChat)에서 확인할 수 있습니다.

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 [mathchat] 옵션을 설치하세요.
```bash
pip install "pyautogen[mathchat]"
```

In [ ]:
# %pip install "pyautogen[mathchat]~=0.1.1"

## API 엔드포인트 설정

`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다.


In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-4-32k-v0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
            "gpt",
        }
    }
)

먼저 유효한 json 문자열이어야 하는 환경 변수 "OAI_CONFIG_LIST"를 찾습니다. 해당 변수를 찾을 수 없으면 "OAI_CONFIG_LIST"라는 이름의 json 파일을 찾습니다. 이 파일은 모델별로 구성을 필터링합니다(다른 키로도 필터링할 수 있음).

설정 목록은 다음과 같습니다:
```python
config_list = [
    {
        '모델': 'gpt-4',
        'api_key': '<귀하의 OpenAI API 키는 여기>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 '파일 업로드' 아이콘을 선택해 파일을 업로드할 수 있습니다.

YAML 파일에서 불러오기 등 원하는 다른 방법으로 config_list의 값을 설정할 수 있습니다.

## 수학 채팅용 에이전트 구축

먼저 `AssistantAgent`와 `MathUserProxyAgent`를 초기화합니다. 수학 채팅의 시스템 메시지는 "당신은 유용한 조수입니다."로 설정해야 합니다. 자세한 지침은 사용자 메시지에 나와 있습니다. 나중에 `MathUserProxyAgent.generate_init_message`를 사용하여 지침과 수학 문제를 결합하여 LLM 어시스턴트에게 전송할 초기 메시지를 생성합니다.

In [ ]:
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent

autogen.ChatCompletion.start_logging()

# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

# 2. create the MathUserProxyAgent instance named "mathproxyagent"
# By default, the human_input_mode is "NEVER", which means the agent will not ask for human input.
mathproxyagent = MathUserProxyAgent(
    name="mathproxyagent", 
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

### 예제 1

문제: 부등식 $(2x+10)(x+3)<(3x+9)(x+8)$을 만족하는 모든 $x$를 구하세요. 답을 구간 표기법으로 표현하세요.

정답을 구합니다:
begin{align*} (2x+10)(x+3)&<(3x+9)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)&<3(x+3)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)-3(x+3)(x+8)&<0 \quad \Rightarrow
\\ (2x+10-(3x+24))(x+3)&<0 \quad \Rightarrow
\\ (-x-14)(x+3)&<0 \quad \Rightarrow
\\ (x+14)(x+3)&>0.
\end{align*} 이 부등식은 $(x+14)$와 $(x+3)$가 모두 양이거나 둘 다 음인 경우에만 만족됩니다.  x>-3$의 경우 두 요인 모두 양수이고, $x<-14$의 경우 두 요인 모두 음수입니다.  14<x<-3$일 때, 한 요인은 양수이고 다른 요인은 음수이므로 이들의 곱은 음수입니다.   따라서 부등식을 만족하는 $x$의 범위는 $ \boxed{(-\인프티, -14)\cup(-3,\인프티)}$입니다.

In [ ]:
# given a math problem, we use the mathproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the mathproxyagent for processing.
# The conversation continues until the termination condition is met, in MathChat, the termination condition is the detect of "\boxed{}" in the response.
math_problem = "Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."
mathproxyagent.initiate_chat(assistant, problem=math_problem)

### 예제 2
문제: $k$의 음수 값에 대해 방정식 \begin{align*}의 계에 정확히 하나의 해가 있습니다.
y &= 2x^2 + kx + 6 \\?
y &= -x + 4?
\end{align*}

정답 y$에 대한 두 식을 서로 같게 설정하면 $2x^2 + kx + 6 = -x + 4$가 됩니다. 다시 정렬하면 $2x^2 + (k+1)x + 2 = 0$이 됩니다. x$에 대한 해가 정확히 하나만 존재하려면 주어진 이차방정식의 미분은 0과 같아야 합니다. 따라서 $(k+1)^2 - 4 \cdot 2 \cdot 2 = (k+1)^2 - 16 = 0$이므로 $k+1 = \pm 4$입니다. 음의 값을 취하면 $k = \boxed{-5}$가 됩니다.

In [ ]:
math_problem = "For what negative value of $k$ is there exactly one solution to the system of equations \\begin{align*}\ny &= 2x^2 + kx + 6 \\\\\ny &= -x + 4?\n\\end{align*}"
mathproxyagent.initiate_chat(assistant, problem=math_problem)

### 예 3
문제: $x^2-7x+c=0$ 방정식의 근이 실수이고 유리한 근만 갖도록 $c$의 모든 양의 정수 값을 구합니다. 쉼표로 구분하여 감소하는 순서로 표현합니다.

정답: 근이 실수이고 합리적이기 위해서는 판별자가 완벽한 제곱이어야 합니다. 따라서 $(-7)^2-4 \cdot 1 \cdot c = 49-4c$는 완전 제곱이어야 합니다. 49보다 작은 양의 완전 제곱은 $1$, $4$, $9$, $16$, $25$, $36$뿐입니다. c$의 정수 값을 제공하는 완전 제곱은 $1$, $9$, $25$입니다. 따라서 $49-4c=1$, $49-4c=9$, $49-4c=25$ 방정식을 갖게 됩니다. 이 방정식을 풀면 c의 양의 정수 값은 $\boxed{12, 10, 6}$임을 알 수 있습니다.

In [ ]:
math_problem = "Find all positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational. Express them in decreasing order, separated by commas."
mathproxyagent.initiate_chat(assistant, problem=math_problem)

------------------------------------
### 다른 프롬프트 사용

MathChat은 어시스턴트에게 문제 해결을 지시하는 다양한 프롬프트를 허용합니다.

MathUserProxyAgent.generate_init_message(문제, 프롬프트 유형='기본', 사용자 지정 프롬프트=None)`를 확인하세요:
- 프롬프트 유형` 매개변수는 `['default', 'python', 'two_tools']` 중에서 선택할 수 있습니다.  이 문서에는 두 가지 프롬프트가 더 포함되어 있습니다:
    1. `'python'`은 기본 프롬프트에서 Python만 사용하는 간소화된 프롬프트입니다.
    2. `'two_tools'는 이 간소화된 `python` 프롬프트에 따라 Python 또는 Wolfram Alpha를 선택할 수 있도록 합니다. 이 옵션을 사용하려면 Wolfram Alpha API 키가 필요하며, `wolfram.txt`에 입력해야 한다는 점에 유의하시기 바랍니다.

- 필요한 경우 사용자 정의 프롬프트를 입력할 수도 있습니다: mathproxyagent.generate_init_message(problem, customized_prompt="사용자 정의 프롬프트")`를 입력할 수 있습니다. 이 수학 프록시 에이전트는 '\boxed{}'를 종료로 감지하므로 프롬프트에 비슷한 종료 문장을 입력해야 합니다: "답을 얻으면 \\boxed{}에 답을 입력하세요.". 사용자 지정이 제공된 경우 '프롬프트 유형'은 무시됩니다.


### 예 4("python" 프롬프트 사용):

문제: $725x + 727y = 1500$이고 $729x + 731y = 1508$일 때 $x - y$의 값은 무엇인가요?

정답은 다음과 같습니다: 두 방정식을 빼면 다음과 같습니다:
\begin{align*}
(729x+731y)-(725x+727y) &= 1508-1500\\.
\Rightarrow\qquad 4x+4y &= 8\\
\Rightarrow\qquad x+y &= 2\\.
\end{align*}

이 방정식에 725를 곱하고 $725x+727y=1500$에서 이 방정식을 빼면 \begin{align*}이 됩니다.
(725x+727y) - 725(x+y) &= 1500-725(x+y) \implies \\.
2y &= 50.
\end{align*}따라서 $x-y$를 $(x+y) - 2y$로 쓸 수 있으며, 이는 $2 - 50 = \boxed{-48}$과 같습니다.


In [ ]:
# we set the prompt_type to "python", which is a simplied version of the default prompt.
math_problem = "Problem: If $725x + 727y = 1500$ and $729x+ 731y = 1508$, what is the value of $x - y$ ?"
mathproxyagent.initiate_chat(assistant, problem=math_problem, prompt_type="python")

## 예제 5("two_tools" 프롬프트 사용)

문제: $y=x^2+a$의 그래프와 $y=ax$의 그래프가 교차하는 모든 수 $a$를 구합니다. 답을 구간 표기법으로 표현하세요.


정답: 이 두 그래프가 교차하는 경우 교차점은 \[x^2+a=ax,\] 또는 \[x^2-ax+a=0.\] 이 이차방정식은 판별자가 음이 아닐 때 정확히 해를 갖습니다: \[(-a)^2-4\cdot1\cdot a\geq0.\] 이것은 \[a(a-4)\geq0.\]로 단순화됩니다. $a$와 $a-4$가 둘 다 $\ge 0$이거나 둘 다 $\le 0$일 때 이 이차방정식($a$에서)은 음이 아닙니다. 이는 $$(-\인프티,0]\컵[4,\인프티)$$의 $a$에 해당합니다.$$ 따라서 $a$가 $\boxed{(-\인프티,0]\컵[4,\인프티)}$에 있을 때 선과 이차방정식은 정확히 교차합니다.


In [ ]:
# The wolfram alpha app id is required for this example (the assistant may choose to query Wolfram Alpha).
import os
if "WOLFRAM_ALPHA_APPID" not in os.environ:
    os.environ["WOLFRAM_ALPHA_APPID"] = open("wolfram.txt").read().strip()

# we set the prompt_type to "two_tools", which allows the assistant to select wolfram alpha when necessary.
math_problem = "Find all numbers $a$ for which the graph of $y=x^2+a$ and the graph of $y=ax$ intersect. Express your answer in interval notation."
mathproxyagent.initiate_chat(assistant, problem=math_problem, prompt_type="two_tools")